In [40]:
cd desktop


c:\Users\dev23\desktop


In [41]:
cd ML

c:\Users\dev23\desktop\ML


In [42]:
cd research

c:\Users\dev23\desktop\ML\research


In [43]:
import os
%pwd

'c:\\Users\\dev23\\desktop\\ML\\research'

In [44]:
os.chdir("../")
%pwd

'c:\\Users\\dev23\\desktop\\ML'

In [45]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/DEVJUNEJA23/ML.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="DEVJUNEJA23"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bcd556e172ca91d8df95d5b41cc7e341ce55f9bc"

In [46]:
import tensorflow as tf

In [47]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [49]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [50]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/DEVJUNEJA23/ML.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [51]:
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [52]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [53]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [54]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e


[2024-01-29 01:58:33,151:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-29 01:58:33,160:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-29 01:58:33,165:INFO:common:created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 12s 1s/step - loss: 29.1907 - accuracy: 0.4820
[2024-01-29 01:58:46,563:INFO:common:json file saved at: scores.json]


2024/01/29 01:58:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\dev23\AppData\Local\Temp\tmpr11vs0_h\model\data\model\assets
[2024-01-29 01:58:53,222:INFO:builder_impl:Assets written to: C:\Users\dev23\AppData\Local\Temp\tmpr11vs0_h\model\data\model\assets]


2024/01/29 01:59:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\dev23\AppData\Local\Temp\tmpr11vs0_h\model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\dev23\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/01/29 02:02:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
